In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Función para convertir el timestamp a fecha legible
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')

# Cargar los datos desde los archivos JSON
ipc_europe = pd.read_json('../docs/investing ipc-mensual-europa-928.json', orient='records')
ipc_eeuu = pd.read_json('../docs/investing_ipc-mensual-eeu-69.json', orient='records')

# Crear DataFrame a partir de la columna 'data' y asignar nombres de columnas
ipc_eeuu_df = pd.DataFrame(ipc_eeuu['data'].to_list(), columns=['timestamp', 'change_ipc_eeuu', 'trend_ipc_eeuu'])
ipc_europe_df = pd.DataFrame(ipc_europe['data'].to_list(), columns=['timestamp', 'change_ipc_europe', 'trend_ipc_europe'])

# Convertir el timestamp a fecha legible para ambos DataFrames
ipc_eeuu_df['date_ipc_eeuu'] = pd.to_datetime(ipc_eeuu_df['timestamp'] / 1000, unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S')
ipc_europe_df['date_ipc_europe'] = pd.to_datetime(ipc_europe_df['timestamp'] / 1000, unit='s').dt.strftime('%Y-%m-%dT%H:%M:%S')

# Ordenar por fecha
ipc_eeuu_df = ipc_eeuu_df.sort_values(by='date_ipc_eeuu').reset_index(drop=True)
ipc_europe_df = ipc_europe_df.sort_values(by='date_ipc_europe').reset_index(drop=True)

# Eliminar la columna 'timestamp'
ipc_eeuu_df.drop(columns=['timestamp'], inplace=True)
ipc_europe_df.drop(columns=['timestamp'], inplace=True)

# Función para rellenar fechas faltantes con interpolación
def fill_missing_dates(df, date_column, change_column):
    # Crear un rango completo de fechas desde la primera hasta la última fecha en el DataFrame
    full_date_range = pd.date_range(start=df[date_column].min(), end=df[date_column].max())
    
    # Crear un DataFrame con el rango completo de fechas
    full_df = pd.DataFrame({date_column: full_date_range})
    
    # Convertir la columna de fecha al tipo datetime64[ns]
    df[date_column] = pd.to_datetime(df[date_column])
    
    # Unir el DataFrame original con el DataFrame de fechas completas
    merged_df = pd.merge(full_df, df, on=date_column, how='left')
    
    # Rellenar los valores de 'change' interpolando
    merged_df[change_column] = merged_df[change_column].interpolate()
    
    return merged_df

# Aplicar la función para rellenar fechas faltantes en ambos DataFrames
filled_ipc_eeuu_df = fill_missing_dates(ipc_eeuu_df, 'date_ipc_eeuu', 'change_ipc_eeuu')
filled_ipc_europe_df = fill_missing_dates(ipc_europe_df, 'date_ipc_europe', 'change_ipc_europe')

# Asignar valores a 'trend' basado en cambios en 'change'
def assign_trend_based_on_change(df, change_column, trend_column):
    df[trend_column] = (df[change_column].diff() > 0).astype(int)
    df[trend_column].mask(df[change_column].diff() == 0, inplace=True)  # Si el cambio es 0, dejar 'trend' como NaN
    df[trend_column].fillna(2, inplace=True)  # Reemplazar NaN con 2
    return df

filled_ipc_eeuu_df = assign_trend_based_on_change(filled_ipc_eeuu_df, 'change_ipc_eeuu', 'trend_ipc_eeuu')
filled_ipc_europe_df = assign_trend_based_on_change(filled_ipc_europe_df, 'change_ipc_europe', 'trend_ipc_europe')

# Guardar los DataFrames en archivos CSV con la fecha en formato ISO con 'T'
filled_ipc_eeuu_df['date_ipc_eeuu'] = filled_ipc_eeuu_df['date_ipc_eeuu'].dt.strftime('%Y-%m-%dT%H:%M:%S')
filled_ipc_europe_df['date_ipc_europe'] = filled_ipc_europe_df['date_ipc_europe'].dt.strftime('%Y-%m-%dT%H:%M:%S')

filled_ipc_eeuu_df.to_csv('../docs/filled_ipc_eeuu.csv', index=False)
filled_ipc_europe_df.to_csv('../docs/filled_ipc_europe.csv', index=False)



# Unir los DataFrames por la columna 'date'
merged_df = pd.merge(filled_ipc_eeuu_df, filled_ipc_europe_df, left_on='date_ipc_eeuu', right_on='date_ipc_europe', how='inner')

# Mantener la columna de fecha de la izquierda y eliminar la de la derecha
merged_df = merged_df.drop(columns=['date_ipc_europe']).rename(columns={'date_ipc_eeuu': 'date'})

# Guardar el DataFrame resultante en un archivo CSV
merged_df.to_csv('../docs/merged_ipc.csv', index=False)

print("Merge complete. The merged dataframe has been saved to '../docs/merged_ipc.csv'")

Merge complete. The merged dataframe has been saved to '../docs/merged_ipc.csv'


In [9]:
import pandas as pd

# Cargar los datos desde los archivos CSV
brent_df = pd.read_csv('../docs/Brent Oil Futures Historical Data.csv')
wti_df = pd.read_csv('../docs/Crude Oil WTI Futures Historical Data.csv')

# Renombrar las columnas para diferenciarlas entre Brent y WTI
brent_df = brent_df.rename(columns=lambda x: f'{x}_brent' if x != 'Date' else 'Date')
wti_df = wti_df.rename(columns=lambda x: f'{x}_wti' if x != 'Date' else 'Date')

# Convertir la columna de fecha a datetime
brent_df['Date'] = pd.to_datetime(brent_df['Date'])
wti_df['Date'] = pd.to_datetime(wti_df['Date'])

# Ordenar por fecha
brent_df = brent_df.sort_values(by='Date').reset_index(drop=True)
wti_df = wti_df.sort_values(by='Date').reset_index(drop=True)

# Unir los DataFrames por la columna 'Date'
merged_df = pd.merge(brent_df, wti_df, on='Date', how='inner')

# Guardar el DataFrame resultante en un archivo CSV
merged_df.to_csv('../docs/merged_oil_futures.csv', index=False)

print("Merge complete. The merged dataframe has been saved to '../docs/merged_oil_futures.csv'")


Merge complete. The merged dataframe has been saved to '../docs/merged_oil_futures.csv'
